<a href="https://colab.research.google.com/github/Sem-Enyo/Zonal-Statistics-Automation-Tool/blob/main/Zonal_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy

import os

import sys

import string

from arcpy import env



In [ ]:
#The name of the input layer to be used
poly = arcpy.GetParameterAsText(0)  

#the field to be used for the statistics       
stat_field = arcpy.GetParameterAsText(1)  

#The statistics type to be calculated  
stat_type = arcpy.GetParameterAsText(2)    

#Getting the input rasters
raster_name = arcpy.GetParameterAsText(3) 
  
#The input rasters are seperated by ';'
#There we need to split it by ';', to create a list for the input rasters
raster_list= raster_name.split(';')   
      
#The order of the rasters needs to be printed as it will be used as  reference in the final output table
arcpy.AddMessage('The order of the raster(s) is shown below: ')    

zone_list = []  #A list that will contain the zonal statistics output 
n = 1           #This number will be used to geerate different names ot the ouput for every iteraation

for raster in raster_list:
    arcpy.AddMessage(raster)    #Printing the name of the raster 
    arcpy.sa.ZonalStatisticsAsTable(poly, stat_field, raster, 'r_' + str(n) + '.dbf' , 'DATA', str(stat_type))
    zone_list.append('r_' + str(n)+ '.dbf')     #Appending the name of the output table into the zone_list
    n+=1
 
#Joining the zonal statistics table to the original polygon based on a common field
intable=poly
in_field = stat_field

#In some case the the whole path of the input polygon is stored into the variable
#Therefore we need to split it by the seperators to extract the name of the input polygon
#The result of the first zonal statistics adds the name of the input polygon to the column names
#This combination is needed to extract the join field when joing the tables together
poly_list= poly.split("\\")
  
if poly_list[-1][-3:] == 'shp' :
    poly_field = str(poly_list[-1][:-3])  + str(stat_field)
    
else:
    poly_field = str(poly_list[-1]) + '.' + str(stat_field)
      
#Iterating throught the output of the zonal statistics to join them into one table                        
for i in range(len(zone_list)):
    result_table = arcpy.management.AddJoin(intable , in_field, zone_list[i], stat_field, 'KEEP_COMMON')
    #The next join will now use the results of the previous join
    intable=result_table  
    in_field = poly_field


#Getting the output table location folder       
outLocation = arcpy.GetParameterAsText(4)    

#Gettiing the name of the final output table
final_table = arcpy.GetParameterAsText(5)    

#Saving the final table in the output folder location
arcpy.conversion.TableToTable(result_table,outLocation, str(final_table) + '.dbf')

arcpy.AddMessage('The final table table is stored in the output folder location selected')
arcpy.AddMessage('The final table columns are in the order of the selected rasters') 


